In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from time import sleep

%matplotlib inline

In [67]:
#Import the dataset
data_train=pd.read_csv('/content/drive/MyDrive/N12-Kĩ thuật giấu tin/CSV/2 patterns (P1-P5)/LẦN 2/Train_560.000.csv')
data_test=pd.read_csv('/content/drive/MyDrive/N12-Kĩ thuật giấu tin/CSV/2 patterns (P1-P5)/LẦN 2/Test_140.000.csv')

X_train=data_train.iloc[:560000,[1,2,3,4,5]].values
Y_train=data_train.iloc[:560000,0].values

X_test=data_test.iloc[:140000,[1,2,3,4,5]].values
Y_test=data_test.iloc[:140000,0].values


In [68]:
X_train, Y_train, X_test, Y_test = list(map(lambda x: np.array(x, dtype=np.float64), [X_train, Y_train, X_test, Y_test]))

In [69]:
data_train.head()

,LABEL,ip.proto,ip.id,tcp.seq_raw,frame.len,frame.cap_len
0,0,6,47010,698643808,1420,66
1,1,6,29184,3590455296,56,56
2,1,6,10496,1845493760,56,56
3,0,6,19580,958742619,66,66
4,0,6,16933,2240452449,54,54


In [70]:
X_train[0]

array([6.00000000e+00, 4.70100000e+04, 6.98643808e+08, 1.42000000e+03,
       6.60000000e+01])

In [84]:
max_words = 432957
max_len = 5
# tok = Tokenizer(num_words=max_words)
# tok.fit_on_texts(X_train)
# sequences = tok.texts_to_sequences(X_train)
#sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
sequences_matrix = X_train
Y_train = Y_train.reshape(-1, 1)

In [72]:
tcp_train = X_train[:, 2]
tcp_test = X_test[:, 2]

In [73]:
#Trộn train+test
tcp = np.concatenate([tcp_train, tcp_test])

In [74]:
tcp.shape

(700000,)

In [75]:
#Sort tcp_test
tcp_set = list(set(tcp))
tcp_sort = list(np.sort(tcp_set))
len(tcp_set)

432957

In [76]:
#Đánh index value tcp_train và tcp_test= index trong tcp+sort
result_train = []
result_test = []
for value in tcp_train:
  index = tcp_sort.index(value)
  result_train.append(index)

for value in tcp_test:
  index = tcp_sort.index(value)
  result_test.append(index)

result_train = np.array(result_train)
result_test = np.array(result_test)

In [85]:
np.max(result_test)

432956

In [86]:
X_train[:, 2] = result_train
X_test[:, 2] = result_test

In [87]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    # layer = Merge([max_len, context_model], mode="dot", dot_axes=0)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [88]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 5)]               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 5, 50)             21647850  
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_8 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [89]:
start = time.time()
model.fit(sequences_matrix,Y_train,batch_size=2048,epochs=10,
          validation_split=0.2)
end = time.time()

Epoch 1/10
219/219 [==============================] - 21s 88ms/step - loss: 0.0489 - accuracy: 0.9864 - val_loss: 9.4790e-04 - val_accuracy: 0.9998
Epoch 2/10
219/219 [==============================] - 19s 85ms/step - loss: 5.2100e-04 - accuracy: 0.9999 - val_loss: 5.8390e-04 - val_accuracy: 0.9999
Epoch 3/10
219/219 [==============================] - 19s 85ms/step - loss: 3.7057e-04 - accuracy: 0.9999 - val_loss: 7.6633e-04 - val_accuracy: 0.9999
Epoch 4/10
219/219 [==============================] - 19s 85ms/step - loss: 2.8220e-04 - accuracy: 1.0000 - val_loss: 5.4999e-04 - val_accuracy: 0.9999
Epoch 5/10
219/219 [==============================] - 19s 85ms/step - loss: 2.4651e-04 - accuracy: 1.0000 - val_loss: 4.8996e-04 - val_accuracy: 0.9999
Epoch 6/10
219/219 [==============================] - 19s 85ms/step - loss: 1.7959e-04 - accuracy: 1.0000 - val_loss: 7.7513e-04 - val_accuracy: 0.9999
Epoch 7/10
219/219 [==============================] - 19s 85ms/step - loss: 1.4402e-04 - acc

In [90]:
exe_time = end - start
print(exe_time)

188.22588920593262


In [91]:
start = time.time()
model.evaluate(X_test, Y_test)
end = time.time()
exe_time = end - start
print(exe_time)

4375/4375 [==============================] - 10s 2ms/step - loss: 9.6711e-04 - accuracy: 0.9999
10.468656301498413


In [94]:
model.predict(np.array([X_test[4]]))

array([[1.]], dtype=float32)

In [95]:
Y_test[4]

1.0